# Quotes Pipeline

In this pipeline architecture a Cloud Scheduler Job is scheduled to run every couple of minutes which pushes a message to a Pub/Sub topic named quote-fetcher-topic. The quote-fetcher-topic Pub/Sub topic in turn invokes a Cloud Function named quote_fetcher which fetches random quotes from https://quotes.toscrape.com/random and, parses them from the webpage HTML and into a JSON datastructure then publishes the resulting JSON message into a second Pub/Sub topic named quotes. The quote Pub/Sub topic is comsumed by an Apache Beam Dataflow pipeline which parses the JSON based quote, calls the Natural Language ML API passing the quote text and receiving back sentiment scores which are added to the quote data structure. The Beam pipeline concludes by saving the sentiment enriched quotes to a BigQuery table named quotes in a quotesds dataset. 

<img src="./Quote-Sentiment-Pipeline.jpg">

## Building the Quote Fetcher Cloud Function

Steps are modified from GCP docs tutorial [Using Pub/Sub to trigger a Cloud Function](https://cloud.google.com/scheduler/docs/tut-pub-sub) along with the Quick Start example for [Functions Framework GitHub README](https://github.com/GoogleCloudPlatform/functions-framework-python)

Quick View of Steps:

1) Create Pub/Sub topic to write quotes to from Quote Fetcher Cloud Function

2) Create Quote Fetcher Cloud Function 

3) Create Pub/Sub topic to trigger Quote Fetcher Cloud Function

4) Create Cloud Scheduler job to invoke Pub/Sub topic


__Step 1:__ Create a pubsub topic for the Quote Fetcher cloud function to publish quotes to.

In [1]:
! gcloud pubsub topics create quotes

Created topic [projects/qwiklabs-gcp-04-2ad6a04dc593/topics/quotes].


__Step 2:__ Create Quote Fetcher Cloud Function

First need a directory to hold the source code in.

In [2]:
%%bash

if [ ! -d quote_fetcher ]
then
  echo "creating quote_fetcher directory"
  mkdir quote_fetcher
fi

if [ ! -d pubsub_schd ]
then
  echo "creating pubsub_schd directory"
  mkdir pubsub_schd
fi

creating quote_fetcher directory


Write requirements.txt for Quote Fetcher Cloud Function Python dependencies

In [3]:
%%writefile ./quote_fetcher/requirements.txt
google-cloud-pubsub==2.7.0
requests>=2.26.0,<2.27.0
beautifulsoup4>=4.9.3,<4.10.0
pydantic>=1.8.2,<1.9.0
# google-cloud-language>=2.2.2,<2.3.0

Writing ./quote_fetcher/requirements.txt


Write Quote Fetcher Cloud Function source code

In [4]:
%%writefile ./quote_fetcher/main.py

import json
import os
import typing

import requests

from bs4 import BeautifulSoup
from google.cloud import pubsub_v1

from pydantic import BaseModel

PROJECT_ID = os.environ['PROJECT_ID']
TOPIC_ID = os.environ['TOPIC_ID']


class Quote(BaseModel):
    text : str
    author : str
    tags : typing.Sequence[str]
    sentiment : typing.Optional[float]
    magnitude : typing.Optional[float]
        

def fetch_quote(events, context):
    quote_url = 'https://quotes.toscrape.com/random'

    response = requests.get(quote_url)

    soup = BeautifulSoup(response.content, 'html.parser')

    quote_el = soup.find('div', class_='quote')

    quote = Quote(
        text=quote_el.find('span', class_='text').get_text(),
        author=quote_el.find('small', class_='author').get_text(),
        tags=[el.get_text() for el in quote_el.find_all('a', class_='tag')]
    )
    
    quote_data = quote.dict()
    print("PROJECT_ID " + PROJECT_ID)
    print("TOPIC_ID " + TOPIC_ID)
    print(quote_data)
    
    publisher = pubsub_v1.PublisherClient()
    topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)
    publisher.publish(topic_path, json.dumps(quote_data).encode('utf-8'))
    
    return quote_data

Writing ./quote_fetcher/main.py


__Step 3:__ Write a helper deployment shell script which also creates a Pub/Sub Topic

In [5]:
%%writefile ./quote_fetcher/deploy-cloud-function.sh

#!/bin/bash

if [ -d quote_fetcher ]
then
  cd quote_fetcher
fi

set -ex

PROJECT_ID=$(gcloud config get-value project)
TOPIC_ID=quotes

gcloud functions deploy quote_fetcher \
  --set-env-vars PROJECT_ID=$PROJECT_ID,TOPIC_ID=$TOPIC_ID \
  --entry-point fetch_quote \
  --runtime python37 \
  --trigger-topic quote-fetcher-topic

Writing ./quote_fetcher/deploy-cloud-function.sh


Deploy the Quote Fetcher Cloud Function 

In [6]:
%%bash

chmod +x quote_fetcher/deploy-cloud-function.sh

./quote_fetcher/deploy-cloud-function.sh

availableMemoryMb: 256
buildId: b5d17b9e-fa1a-4d63-ae57-6e3ef5f41c9c
buildName: projects/774131484409/locations/us-central1/builds/b5d17b9e-fa1a-4d63-ae57-6e3ef5f41c9c
entryPoint: fetch_quote
environmentVariables:
  PROJECT_ID: qwiklabs-gcp-04-2ad6a04dc593
  TOPIC_ID: quotes
eventTrigger:
  eventType: google.pubsub.topic.publish
  failurePolicy: {}
  resource: projects/qwiklabs-gcp-04-2ad6a04dc593/topics/quote-fetcher-topic
  service: pubsub.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/qwiklabs-gcp-04-2ad6a04dc593/locations/us-central1/functions/quote_fetcher
runtime: python37
serviceAccountEmail: qwiklabs-gcp-04-2ad6a04dc593@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-0590737b-324c-4a57-b58b-fd974ee68e4f/54316c8d-fca6-4019-b329-11720ec78181.zip
status: ACTIVE
timeout: 60s
updateTime: '2021-08-21T02:07:38.331Z'
versionId: '1'


+++ gcloud config get-value project
++ PROJECT_ID=qwiklabs-gcp-04-2ad6a04dc593
++ TOPIC_ID=quotes
++ gcloud functions deploy quote_fetcher --set-env-vars PROJECT_ID=qwiklabs-gcp-04-2ad6a04dc593,TOPIC_ID=quotes --entry-point fetch_quote --runtime python37 --trigger-topic quote-fetcher-topic
Deploying function (may take a while - up to 2 minutes)...
..
For Cloud Build Logs, visit: https://console.cloud.google.com/cloud-build/builds;region=us-central1/b5d17b9e-fa1a-4d63-ae57-6e3ef5f41c9c?project=774131484409
.................................................done.


Publish some data to the quote-fetcher-topic Pub/Sub topic

In [11]:
! gcloud pubsub topics publish quote-fetcher-topic --message "this is a test message"

messageIds:
- '2906104492847693'


Check out the logs to make sure the Quote Fetcher Cloud Function is being Fired by Pub/Sub events

In [12]:
! gcloud functions logs read quote_fetcher --limit 12

LEVEL  NAME           EXECUTION_ID  TIME_UTC                 LOG
D      quote_fetcher  v2mpufvuid6d  2021-08-21 02:07:56.710  Function execution took 487 ms, finished with status: 'ok'
I      quote_fetcher  v2mpufvuid6d  2021-08-21 02:07:56.485  {'text': '“You can never get a cup of tea large enough or a book long enough to suit me.”', 'author': 'C.S. Lewis', 'tags': ['books', 'inspirational', 'reading', 'tea'], 'sentiment': None, 'magnitude': None}
I      quote_fetcher  v2mpufvuid6d  2021-08-21 02:07:56.485  TOPIC_ID quotes
I      quote_fetcher  v2mpufvuid6d  2021-08-21 02:07:56.485  PROJECT_ID qwiklabs-gcp-04-2ad6a04dc593
D      quote_fetcher  v2mpufvuid6d  2021-08-21 02:07:56.223  Function execution started
D      quote_fetcher  v2mp1emgp6j7  2021-08-21 02:07:56.195  Function execution took 4026 ms, finished with status: 'ok'
I      quote_fetcher  v2mp1emgp6j7  2021-08-21 02:07:55.559  {'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The o

__Step 4:__ Create a Cloud Schedule Job to Push Messages to Pub/Sub which in turn Invokes the Cloud Function

In cloud shell runt he following.

```sh
gcloud services enable cloudscheduler.googleapis.com

export PROJECT_ID=$(gcloud config get-value project)
gcloud app create --project $PROJECT_ID --region us-central

gcloud scheduler jobs create pubsub quotefetcher \
  --schedule "*/1 * * * *" \
  --topic quote-fetcher-topic \
  --message-body "fetch quote"
```

## Create a Dataflow Pipeline

Create a directory to hold Beam Pipeline code

In [13]:
! mkdir quote_pipeline

Create a requriements.txt file for Beam Python library dependencies

In [14]:
%%writefile ./quote_pipeline/requirements.txt

google-cloud-language==2.2.2

Writing ./quote_pipeline/requirements.txt


Below is the pipeline code which consumes data from Pub/Sub quotes topic containing messages in JSON format as shown below.

```json
{
  "text": "Contents of a quote",
  "author": "The Person Attributed with the Quote",
  "tags": ['A', 'list', 'of', 'tags', 'associated', 'with', 'quote'], 
  
}
```

In [20]:
%%writefile ./quote_pipeline/pipeline.py

import argparse
import typing

import apache_beam as beam
from apache_beam.io.gcp.internal.clients import bigquery
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import StandardOptions, SetupOptions
from apache_beam.runners import DataflowRunner

import google.auth
from google.cloud import language

import time

import json


class Quote(typing.NamedTuple):
    text : str
    author : str
    tags : typing.Sequence[str]
    sentiment : float
    magnitude : float

beam.coders.registry.register_coder(Quote, beam.coders.RowCoder)


def analyze_quote(element):
    row = json.loads(element.decode('utf-8'))
    
    client = language.LanguageServiceClient()
    
    doc = language.Document(content=row['text'],
                            type_=language.Document.Type.PLAIN_TEXT)
    
    response = client.analyze_sentiment(document=doc)

    row.update(
      sentiment=response.document_sentiment.score,
      magnitude=response.document_sentiment.magnitude
    )
    
    return row


def main(args, beam_args):
    options = PipelineOptions(beam_args,
                              runner=args.runner,
                              streaming=True,
                              project=args.project,
                              region=args.region,
                              job_name='{}{}'.format('quotes-pipeline-', time.time_ns()),
                              staging_location=args.staginglocation,
                              temp_location=args.templocation,
                              save_main_session=True)
    
    table_spec = bigquery.TableReference(projectId=args.project,
                                         datasetId=args.bqdataset,
                                         tableId=args.bqtable)
    QUOTES_TABLE_SCHEMA = {
        "fields": [
            {
                "name": "text",
                "type": "STRING"
            },
            {
                "name": "author",
                "type": "STRING"
            },
            {
                "name": "tags",
                "type": "STRING",
                "mode": "REPEATED"
            },
            {
                "name": "sentiment",
                "type": "FLOAT"
            },
            {
                "name": "sentiment",
                "type": "FLOAT"
            }
        ]
    }
    
    with beam.Pipeline(options=options) as p:
        (p  | "ReadPubSub" >> beam.io.ReadFromPubSub(args.pubsubtopic)
            | "AnalyzeQuote" >> beam.Map(analyze_quote)
            | "SaveToBigQuery" >> beam.io.WriteToBigQuery(
                                          table_spec,
                                          schema=QUOTES_TABLE_SCHEMA,
                                          create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                          write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND))

        
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--runner', default='DataflowRunner')
    parser.add_argument('--project')
    parser.add_argument('--region')
    parser.add_argument('--bqdataset')
    parser.add_argument('--bqtable')
    parser.add_argument('--staginglocation')
    parser.add_argument('--templocation')
    parser.add_argument('--pubsubtopic')
    parser.add_argument('--requirements_file')
    
    args, beam_args = parser.parse_known_args()
    
    main(args, beam_args)

Overwriting ./quote_pipeline/pipeline.py


Create Staging and Temp Dataflow Cloud Storage Endpoints

In [16]:
%%bash

export PROJECT_ID=$(gcloud config get-value project)
PIPELINE_BUCKET="gs://quotes-pipeline-$PROJECT_ID"
gsutil mb -l US $PIPELINE_BUCKET

Creating gs://quotes-pipeline-qwiklabs-gcp-04-2ad6a04dc593/...


In [ ]:
%%bash
export PROJECT_ID=$(gcloud config get-value project)
PIPELINE_BUCKET="gs://quotes-pipeline-$PROJECT_ID"
python quote_pipeline/pipeline.py \
   --project $PROJECT_ID \
   --region us-central1 \
   --bqdataset quotesds \
   --bqtable quotes \
   --staginglocation $PIPELINE_BUCKET/staging \
   --templocation $PIPELINE_BUCKET/temp \
   --pubsubtopic projects/qwiklabs-gcp-04-2ad6a04dc593/topics/quotes \
   --requirements_file quote_pipeline/requirements.txt 

Error from last run 

Error message from worker: generic::unknown: Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1239, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 588, in apache_beam.runners.common.SimpleInvoker.invoke_process
  File "/opt/conda/lib/python3.7/site-packages/apache_beam/transforms/core.py", line 1592, in <lambda>
    wrapper = lambda x: [fn(x)]
  File "quote_pipeline/pipeline.py", line 35, in analyze_quote
AttributeError: module 'google.cloud.language' has no attribute 'Document'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py", line 289, in _execute
    response = task()
  File "/usr/local/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py", line 362, in <lambda>
    lambda: self.create_worker().do_instruction(request), request)
  File "/usr/local/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py", line 607, in do_instruction
    getattr(request, request_type), request.instruction_id)
  File "/usr/local/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py", line 644, in process_bundle
    bundle_processor.process_bundle(instruction_id))
  File "/usr/local/lib/python3.7/site-packages/apache_beam/runners/worker/bundle_processor.py", line 1000, in process_bundle
    element.data)
  File "/usr/local/lib/python3.7/site-packages/apache_beam/runners/worker/bundle_processor.py", line 228, in process_encoded
    self.output(decoded_value)
  File "apache_beam/runners/worker/operations.py", line 357, in apache_beam.runners.worker.operations.Operation.output
  File "apache_beam/runners/worker/operations.py", line 359, in apache_beam.runners.worker.operations.Operation.output
  File "apache_beam/runners/worker/operations.py", line 221, in apache_beam.runners.worker.operations.SingletonConsumerSet.receive
  File "apache_beam/runners/worker/operations.py", line 718, in apache_beam.runners.worker.operations.DoOperation.process
  File "apache_beam/runners/worker/operations.py", line 719, in apache_beam.runners.worker.operations.DoOperation.process
  File "apache_beam/runners/common.py", line 1241, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 1321, in apache_beam.runners.common.DoFnRunner._reraise_augmented
  File "/usr/local/lib/python3.7/site-packages/future/utils/__init__.py", line 446, in raise_with_traceback
    raise exc.with_traceback(traceback)
  File "apache_beam/runners/common.py", line 1239, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 588, in apache_beam.runners.common.SimpleInvoker.invoke_process
  File "/opt/conda/lib/python3.7/site-packages/apache_beam/transforms/core.py", line 1592, in <lambda>
    wrapper = lambda x: [fn(x)]
  File "quote_pipeline/pipeline.py", line 35, in analyze_quote
AttributeError: module 'google.cloud.language' has no attribute 'Document' [while running 'AnalyzeQuote-ptransform-42']

passed through:
==>
    dist_proc/dax/workflow/worker/fnapi_service_impl.cc:644
Hide log summary

